<a href="https://colab.research.google.com/github/Midmost/AI/blob/master/EDA_gu_0511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean District in Seoul budget EDA

## Dependencies

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install --upgrade pandas

## Basic information

In [20]:
import pandas as pd
import numpy as np
from pandas.io.formats.info import DataFrameInfo


# excel path
xlsx_file_path = "/content/All_district.xlsx" # TODO: 폴더를 OS로 읽고 한 번에 읽게

# pure excel to pure dataframe and information with a visual-table
df = pd.read_excel(xlsx_file_path)
DataFrameInfo


pandas.io.formats.info.DataFrameInfo

용량이 워낙 크다보니 부분 활용을 하려고 한다

In [40]:
df = df.iloc[0:2000]

매번 새로 불러오기는 너무 무거우니까 잘라놓은 것을 반복해서 사용하자

In [41]:
df_subset = df.iloc[0:2000]
df = df_subset

## Pre-processing

#### pict01 

- [ ]  A규칙: `acct2`열에는 `2)재정계획심의위원회 참석 수당` 와 같은 값이 있다
- [ ]  B규칙: `acct2`열에는 `70,000원*12명*5회*80%` 와 같은 형식의 값이 있다
- [ ]  C규칙: A와 B를 만족하면서 이 둘에 해당하는 값이 concat되어 i행에 있어야한다.
- [ ]  if B규칙에 해당하는 값이 i+1행에 있는 지 확인
- [ ]  true 라면  i행에 있는 값과 concat한다
- [ ]  i+1 행에 있는 값은 지운다
- [ ]  A_1 규칙: `budget22`열에는 `3360` 와 같은 형식의 값이 있어야 한다
- [ ]  if A_1 규칙에 해당하는 행을 i+1행이라고 한다면 i행이 NULL인지 확인한다
- [ ]  true라면 i+1행의 값을 i행으로 replace한다

But mask_B isn't fit with my intent, so I changed rule B

'*' 또는 '원' 또는 '명' 또는 '회' 또는 '%' 거나 숫자 이 중 하나라도 포함되는 경우는 정규식으로

##### 예시

In [23]:
import re

# Define the regular expression pattern
pattern = r'[*원명회%,]'

# Test some strings
print(bool(re.search(pattern, '70,000원*12명*5회*80%'))) # True
print(bool(re.search(pattern, 'abcdefg'))) # False

True
False


#### 규칙 B를 적용

그 전에 NaN 이 있는 지 확인하고 있다면 "exception"이란 이름으로 저장하자

In [34]:
# is there any NaN value in 'bizdetail'?

# Check if there are any NaN values in the 'bizdetail' column
has_nan = df['bizdetail'].isnull().any()

if has_nan:
    print("There are NaN values in the 'bizdetail' column.")
else:
    print("There are no NaN values in the 'bizdetail' column.")

# Count the number of NaN values in the 'bizdetail' column
num_nan = df['bizdetail'].isnull().sum()

print(f"There are {num_nan} NaN values in the 'bizdetail' column.")

# Replace NaN values in the 'bizdetail' column with the string "exception"
df['bizdetail'] = df['bizdetail'].fillna("exception")

There are no NaN values in the 'bizdetail' column.
There are 0 NaN values in the 'bizdetail' column.


Check the mask whether it is made properly or not. 

In [42]:
# Find rows where the 'acct2' column contains at least one of the characters '*', '원', '명', '회', '%' or a ','
mask = df['detailamt'].str.contains(r'[*원,월식명회%]')

# Replace NaN values in the mask with False
mask = mask.fillna(False)

# Select the rows where the mask is True
df_subset_2 = df[mask]
df_subset_2

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
5,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정활동 홍보 대형 현수막,NaN,"3,700,000원*3회",11100.0,NaN,NaN,6
7,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"1)의정활동 홍보 광고비835,000원*70회",NaN,"835,000원*70회",58450.0,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"1)시설부대비69,600,000원*1식",NaN,"69,600,000원*1식",69600.0,NaN,NaN,1994
1994,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)소규모지하안전영향평가비,NaN,"69,000,000원*1.4",96600.0,NaN,NaN,1995
1996,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)각종 본인증 용역 및 수수료,NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1997
1998,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,406 기타자본이전,본예산,01 기타자본이전,01 기타자본이전,"450,000,000원*1식",450000.0,0.0,450000,1999


23행 같은 경우는 detailamt와 budget22가 한 칸 위로 옮겨져서는 안 된다. 

따라서 규칙을 추가해줘야 한다. 

만약 i행이 bizdetail에서 1)로 시작하는 경우이면서 budget22가 값이 있고 i+1행이 2)로 시작한다면 i행의 detailamt와 budget22 값은 한 칸 위로 옮겨져서는 안 된다. 

In [28]:
# # Find the indices of the rows where the mask is False
# indices = df[~mask].index

# # Shift the values of the "detailamt" and "budget22" columns one cell up for these rows
# df.loc[indices, ["detailamt", "budget22"]] = df.loc[indices + 1, ["detailamt", "budget22"]].values
# df

In [43]:
# mask_A = df['bizdetail'].str.contains(r'\d+\).+')

# Create a boolean mask where True indicates that the cell value starts with '1)'
mask_1 = df['bizdetail'].str.startswith('1)')

# Create a boolean mask where True indicates that the cell value starts with '2)'
mask_2 = df['bizdetail'].str.startswith('2)')

# Find the indices of the rows where the mask is False
indices = df[~mask].index

# Iterate over the indices
for i in indices:
    # Check if the current row satisfies the conditions
    if not (mask_1[i] and pd.notnull(df.at[i, 'budget22']) and mask_2[i + 1]):
        # Shift the values of the "detailamt" and "budget22" columns one cell up for this row
        df.loc[i, ["detailamt", "budget22"]] = df.loc[i + 1, ["detailamt", "budget22"]].values

In [44]:
df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
0,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,NaN,01 사무관리비,01 사무관리비,NaN,69500.0,193770.0,"△38,870",1
1,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정 홍보물 제작,NaN,NaN,69500.0,NaN,NaN,2
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"1)시설부대비69,600,000원*1식",NaN,"69,600,000원*1식",69600.0,NaN,NaN,1994
1994,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)소규모지하안전영향평가비,NaN,"69,000,000원*1.4",96600.0,NaN,NaN,1995
1996,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)각종 본인증 용역 및 수수료,NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1997
1998,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,406 기타자본이전,본예산,01 기타자본이전,01 기타자본이전,"450,000,000원*1식",450000.0,0.0,450000,1999


이제 i+1 행을 다 지워주면 된다. 

mask를 충족시키면서 'bizdetail'과 'detailamt' 의 값이 같으면 해당 row를 지워주자

In [38]:
# Create a boolean mask where True indicates that the values in the 'bizdetail' and 'detailamt' columns are equal
mask_equal = df['bizdetail'] == df['detailamt']

# Combine the two masks using the & operator
combined_mask = mask & mask_equal

# Drop the rows where the combined mask is True
df = df[~combined_mask]

In [39]:
df

,gov,dept,policy,danwi,task,acct1,acct2,bizdetail,bizdetailadd,detailamt,budget22,budget21,budgetdiff,idx
0,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,NaN,01 사무관리비,01 사무관리비,NaN,69500.0,193770.0,"△38,870",1
1,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,1)의정 홍보물 제작,NaN,NaN,69500.0,NaN,NaN,2
2,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 달력 제작11,000원*1,500부*1회",2)의회 달력 제작,"11,000원*1,500부*1회",16500.0,NaN,NaN,3
3,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)강남구의회보 제작13,000,000원*1회",2)강남구의회보 제작,"13,000,000원*1회",13000.0,NaN,NaN,4
4,강남구,구의회사무국,의정활동 지원(일반공공행정/입법및선거관리),의회 홍보 및 의정활동 활성화 지원,의정활동 홍보,201 일반운영비,본예산,"2)의회 홍보물품 구매40,000,000원*1식",2)의회 홍보물품 구매,"40,000,000원*1식",40000.0,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,"1)시설부대비69,600,000원*1식",NaN,"69,600,000원*1식",69600.0,NaN,NaN,1994
1994,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)소규모지하안전영향평가비,NaN,"69,000,000원*1.4",96600.0,NaN,NaN,1995
1996,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,401 시설비및부대비,본예산,1)각종 본인증 용역 및 수수료,NaN,"60,000,000원*1.4",84000.0,NaN,NaN,1997
1998,강남구,주민자치과,주민참여 및 자치역량 강화(일반공공행정/일반행정),문화센터 건립 및 시설개보수,일원1동 복합문화센터 건립,406 기타자본이전,본예산,01 기타자본이전,01 기타자본이전,"450,000,000원*1식",450000.0,0.0,450000,1999


12행이 "1)강남구의회 유튜브, 의원별 의정홍보 영" 

13행이 "상 제작200,00원*23명*1회" 

이렇게 되어있는데 이 경우에도 한 줄로 합쳐야 한다. 

또한 

237행에 "144,499,000원*1/12월*1/30일*84%*86%*20" 

238행에 "일" 

이렇게만 되어있는데 이 경우에도 두 행을 하나로 합쳐야 한다. 

bizdetail 같을 경우 더해줘야 하고 

bizdetailadd 와 detailamt 그리고 budget22, budget21, budgetdiff 는 NaN인 경우에는 replace 아니라면 더하기




## Export

In [ ]:
df = df.reset_index(drop=True) # to_excel에서 index를 버릴 수 있지만 혹시 모르니 넣어둠

# Export the DataFrame to an Excel file
df.to_excel('gangbuk_gu.xlsx', index=False)